In [1]:
import sys
sys.path.insert(0, '../' )

In [2]:
from utils import randombytes as rd

rd.randombytes(16)

b'K57\xfc\xe7\xc6\xe1z\x91\xdc\x16\xf7\xf9|x\x96'

In [3]:
from libgf264fft import clib_wrapper as cgf

cgf.gf264_mul( 1 ,1 )

1

In [4]:
from utils import gf2192 as gf

gf.GF_EXT

3

In [5]:
# the commit scheme

from commit import merkeltreecommit as mt


In [6]:
# challenge

from utils import hash as H


In [7]:
#def _gf_fft( fi , rho , offset ):
#    fi_len = len(fi)
#    assert 0==(fi_len&(fi_len-1))
#    fi_gf264 = list( zip( *[gf.to_gf264s(e) for e in fi] ) )  # transform gflist to 3 gf264lists
#    r_gf264 = [ [] for _ in range(gf.GF_EXT) ]
#    for j in range(gf.GF_EXT):     # 3 x gf264 ffts
#         for k in range(rho): r_gf264[j].extend( cgf.btfy( fi_gf264[j] , offset ^ (k*fi_len) ) )
#    r0 = [ gf.from_gf264s(*e) for e in zip(*r_gf264) ] # transform 3 gf264lists to gflist
#    return r0

#### test #######

offset = 16

fi = [ i+(i<<64)+(i<<128) for i in range(7,9) ]

print( "fi" , [hex(i) for i in fi] )

r = gf.fft( fi , 4 , offset )

#print( "r" , r )
print( "r" , [ hex(i) for i in r] )


fi ['0x700000000000000070000000000000007', '0x800000000000000080000000000000008']
r ['0xcb9ea337ae4e8620cb9ea337ae4e8620cb9ea337ae4e8620', '0xcb9ea337ae4e8628cb9ea337ae4e8628cb9ea337ae4e8628', '0x5d717ce9218663005d717ce9218663005d717ce92186630', '0x5d717ce9218663805d717ce9218663805d717ce92186638', '0xc7919d84d4b13af7c7919d84d4b13af7c7919d84d4b13af7', '0xc7919d84d4b13affc7919d84d4b13affc7919d84d4b13aff', '0x9d8297de8e7dae709d8297de8e7dae709d8297de8e7dae7', '0x9d8297de8e7daef09d8297de8e7daef09d8297de8e7daef']


In [8]:
#def _ibtfy_1( vi , offset ):
#    l = len(vi)
#    vi_gf264 = list( zip( *[gf.to_gf264s(e) for e in vi] ) )
#    r_gf264 = [ [] for _ in range(gf.GF_EXT) ]
#    for j in range(gf.GF_EXT):
#         for k in range( 0,l,2 ):
#              r_gf264[j].extend( cgf.ibtfy( vi_gf264[j][k:k+2] , offset^k ) )
#    r = [ gf.from_gf264s(*e) for e in list( zip(*r_gf264) ) ]
#    return r

#### test #######

print( "r0", [hex(i) for i in r] )

r1 = gf.ibtfy_1( r , offset )
print( "r1", [hex(i) for i in r1] )



r0 ['0xcb9ea337ae4e8620cb9ea337ae4e8620cb9ea337ae4e8620', '0xcb9ea337ae4e8628cb9ea337ae4e8628cb9ea337ae4e8628', '0x5d717ce9218663005d717ce9218663005d717ce92186630', '0x5d717ce9218663805d717ce9218663805d717ce92186638', '0xc7919d84d4b13af7c7919d84d4b13af7c7919d84d4b13af7', '0xc7919d84d4b13affc7919d84d4b13affc7919d84d4b13aff', '0x9d8297de8e7dae709d8297de8e7dae709d8297de8e7dae7', '0x9d8297de8e7daef09d8297de8e7daef09d8297de8e7daef']
r1 ['0x700000000000000070000000000000007', '0x800000000000000080000000000000008', '0x700000000000000070000000000000007', '0x800000000000000080000000000000008', '0x700000000000000070000000000000007', '0x800000000000000080000000000000008', '0x700000000000000070000000000000007', '0x800000000000000080000000000000008']


In [11]:
def _dummy( *params ) : return None

In [24]:
def ldt_commit_phase( f0 , h_state , RS_rho=8 , verbose = 1 ):
    if 1 == verbose : dump = print
    else : dump = _dummy

    poly_len = len(f0)
    assert 0==(poly_len&(poly_len-1)) , 'poly_len is not a power of 2'
    rho = RS_rho
    offset = 1<<63
    dump( f"rho: {rho}, offset: {hex(offset)}" )

    commits = []
    mktrees = []

    dump( f"original poly {f0}")
    dump( "\n### commit phase ###" )

    vi = gf.fft( f0 , rho , offset )
    dump( f"Assume the evaluate on [{hex(offset)},|{poly_len}|x{rho}) has been committed ?" )
    dump( "vi:" , vi )
    dump( f"do a redundent commit here for checking correctness" )
    i = 0
    xi = gf.from_bytes( H.gen( h_state , bytes([3+i,1]) )[:gf.GF_BSIZE] )
    dump( f"derive x0 <- H( h_state || 3 || 1 ): {hex(xi)}" )
    while( 2 < poly_len ):
        dump( f"iteration {i}: [{poly_len}] -> [{poly_len//2}]:" )
        mesg = [ gf.to_bytes(vi[j]) + gf.to_bytes(vi[j+1]) for j in range(0,len(vi),2) ]
        root , randomness , tree = mt.commit( mesg )
        mktrees.append( (root,mesg,randomness,tree) )
        commits.append( root )
        dump( f"commit evaluated valuse. --> commits: [{len(commits)}] <--  #mesg: {len(mesg)}" )

        h_state = H.gen( h_state , gf.to_bytes(xi) )
        dump( f"update h_state <- H( h_state|| xi ): {h_state}" )

        i = i+1
        dump( f"update i: {i}" )
        xi = gf.from_bytes( H.gen( h_state , bytes([3+i,1]) )[:gf.GF_BSIZE] )
        dump( f"derive new challenge xi <- H(h_state||{3+i}||1) : {hex(xi)}" )
        dump( f"deriving new polynomial of length [{poly_len//2}]" )

        vi = gf.ibtfy_1( vi , offset )
        dump( "ibtfy_1:" , hex(offset) , [hex(e) for e in vi] )
        vi_e = vi[::2]
        vi_o = vi[1::2]
        vi = [ vi_e[j]^gf.mul(vi_o[j],xi) for j in range(len(vi_e)) ]
        dump( "n_vi:" , [hex(e) for e in vi] )
        dump( f"evaluated values generated by 1 stage of ibtfy  [{len(vi)}]" )
        offset = offset >> 1
        poly_len = poly_len//2
    cc = gf.ibtfy_1( vi , offset )
    dump( "ibtfy_1:" , hex(offset) , [hex(e) for e in cc] )
    dump( "cc:" , [hex(i) for i in cc ] )
    dump( f"commit deg 1 poly: {hex(cc[0])} + x* {hex(cc[1])}" )
    d1poly = gf.to_bytes(cc[0]) + gf.to_bytes(cc[1])
    h_state = H.gen( gf.to_bytes(xi) , d1poly )
    dump( f"update h_state <- H( xi || c0 || c1 ): {h_state}" )
    return commits , d1poly , mktrees , h_state


In [25]:
def _n_commit( f_length ):
    i = 0
    while( f_length > 2 ):
        i = i+1
        f_length = f_length//2
    return i

def ldt_query_phase( f_length , mktrees, h_state , Nq , RS_rho=8 , verbose = 1 ):
    if 1 == verbose : dump = print
    else : dump = _dummy
    assert Nq < 256 , "need to modify hash inputs if supporting >= 256 queries."

    dump( "\n### query phase ###" )
    queries = [ H.gen(h_state,bytes( [ 3+_n_commit(f_length)+1 , j ] ))[:4] for j in range(1,Nq+1) ]   # use 32 bits of hash results only
    idx_mask = (RS_rho*f_length//2)-1
    queries = [ int.from_bytes(e,'little')&idx_mask for e in queries ]
    dump( f"Queries: [{len(queries)}], {queries}" )

    open_mesgs = []
    j = 0
    for root , all_mesg, randomness, tree in mktrees :
        dump( f"open iteration: {j}" )
        open_mesgs.append( [ mt.open(all_mesg[queries[i]],queries[i],randomness,tree) for i in range(Nq) ] )
        dump( f"proof len:[{len(open_mesgs)}] : auth path len:{len(open_mesgs[-1][0])}" )
        queries = [ q//2 for q in queries ]
        j = j+1
    return open_mesgs


In [26]:
def ldt_gen_proof( f0 , h_state , Nq = 26 , RS_rho = 8 , verbose = 1 ):
    commits , d1poly , mktrees , h_state = ldt_commit_phase( f0 , h_state , RS_rho , verbose )
    open_mesgs = ldt_query_phase( len(f0) , mktrees , h_state , Nq , RS_rho , verbose )
    commits.append( d1poly )
    commits.extend( open_mesgs )
    return commits

###########  test   #############

proof = ldt_gen_proof( [1,2,3,4,5,6,7,8] , b'123' )


rho: 8, offset: 0x8000000000000000
original poly [1, 2, 3, 4, 5, 6, 7, 8]

### commit phase ###
Assume the evaluate on [0x8000000000000000,|8|x8) has been committed ?
vi: [1246866497807618795, 4265964599157179256, 1694739141090504046, 1565830132839253768, 13588509979623779941, 996222563803066535, 334356061807418613, 11359377728264780234, 16458911052986449620, 16948388880601032284, 3219752816587853438, 14265852649938963731, 4519790617497040035, 7537718367003295669, 11908198698919436642, 1542795312697520537, 18373502611120147767, 2528080870869800415, 12307884686738983143, 6012030690928285485, 15765636662194813108, 5932476764803616533, 4168554421296933529, 10738872398854773266, 5972401413023563584, 5182919073939605189, 6053074681583170478, 12324407368897735961, 5726952955255770812, 14790680209824253119, 1403999725530152401, 8882118986198274280, 11385416365251370435, 14089681160904999687, 15854840660061616710, 4903354113959966465, 12730854449350105515, 10522658688429308584, 125088865731970

In [27]:
def ldt_gen_challenges( _poly_len , h_state , commits , d1poly , Nq , RS_rho = 8 , verbose = 1 ):
    if 1 == verbose : dump = print
    else : dump = _dummy

    dump( "######## recovery hash state and challenges ########" )
    poly_len = _poly_len
    i = 0
    xi = []
    xi.append( gf.from_bytes( H.gen( h_state , bytes([3+i,1]) )[:gf.GF_BSIZE] ) )
    dump( f"derive x0 <- H( h_state || 3 || 1 ): {hex(xi[0])}" )
    while( 2 < poly_len ):
        dump( f"iteration {i}: [{poly_len}] -> [{poly_len//2}]:" )
        dump( f"mt.root = commits[{i}] = " ,  commits[i] )

        h_state = H.gen( h_state , gf.to_bytes(xi[i]) )
        dump( f"update h_state <- H( h_state|| xi ): {h_state}" )

        i = i+1
        dump( f"update i: {i}" )
        xi.append( gf.from_bytes( H.gen( h_state , bytes([3+i,1]) )[:gf.GF_BSIZE] ) )
        dump( f"derive new challenge xi <- H(h_state||{3+i}||1) : {hex(xi[i])}" )

        dump( f"new polynomial length [{poly_len//2}]" )
        poly_len = poly_len//2
    h_state = H.gen( gf.to_bytes(xi[i]) , d1poly )
    dump( f"update h_state <- H( xi || deg1poly ): {h_state}" )

    dump( "\n### query phase ###" )
    queries = [ H.gen(h_state,bytes( [ 3+i+1 , j ] ))[:4] for j in range(1,Nq+1) ]
    idx_mask = (RS_rho*_poly_len//2)-1
    queries = [ int.from_bytes(e,'little')&idx_mask for e in queries ]
    dump( f"Queries: [{len(queries)}], {queries}" )
    return xi , queries


In [28]:
def ldt_verify_proof( commits , d1poly , open_mesgs , xi , queries , Nq , verbose = 1 ):
    if 1 == verbose : dump = print
    else : dump = _dummy

    dump( "#### check linear relations and opened commit ######" )
    offset = 1<<63
    j = 0
    for auths in open_mesgs :
        dump( f"open iteration: {j}" )
        dump( f"auths[{len(auths[0])}]:",auths[0] )
        verify_j = [ mt.verify( commits[j] , queries[i] , auths[i] ) for i in range(Nq)]
        dump( f"verify opened commits:" , verify_j )
        if not all(verify_j) : return False

        # check linear relations
        mesg = [ path[0] for path in auths ]
        if 0!=j :
            verify_j = [ check_linear_relation(last_mesg[i],mesg[i],queries[i],xi[j],offset) for i in range(Nq) ]
            dump( f"check linear relations:" , verify_j )
            if not all(verify_j) : return False
        last_mesg = mesg
        queries = [ q//2 for q in queries ]
        offset >>= 1
        j = j+1
    # check deg 1 poly
    verify_j = [ check_deg1poly_linear_relation(last_mesg[i],d1poly,queries[i],xi[-1],offset) for i in range(Nq) ]
    dump( f"check last linear relations (with the d1poly):" , verify_j )
    if not all(verify_j) : return False
    return True

def check_linear_relation( mesgjm1 , mesgj , idx , xi , offset ) :
    cc = gf.ibtfy_1( gf.from_bytes_x2( mesgjm1 ) , offset^(idx<<1) )
    idx &= 1
    cc0 = gf.from_bytes( mesgj[ idx*gf.GF_BSIZE: idx*gf.GF_BSIZE + gf.GF_BSIZE] )
    return cc0 != cc[0]^gf.mul(cc[1],xi)

def check_deg1poly_linear_relation( mesgjm1 , d1poly , idx , xi , offset ) :
    m0 = gf.fft( gf.from_bytes_x2(d1poly) , 1 , (offset>>1)^idx )
    return check_linear_relation( mesgjm1 , gf.to_bytes(m0[0])+gf.to_bytes(m0[1]) , idx , xi , offset )


In [30]:
def ldt_verify( proof , _poly_len , h_state , Nq = 26 , RS_rho = 8 , verbose = 1 ):
    n_commits = _n_commit( _poly_len )
    commits     = proof[:n_commits]
    d1poly      = proof[n_commits]
    open_mesgs  = proof[n_commits+1:]
    xi, queries = ldt_gen_challenges(_poly_len,h_state,commits,d1poly,Nq, RS_rho, verbose )
    return ldt_verify_proof(commits,d1poly,open_mesgs,xi,queries,Nq,verbose)

############# test  ##############

ldt_verify( proof , 8 , b'123' )



######## recovery hash state and challenges ########
derive x0 <- H( h_state || 3 || 1 ): 0x5b9610dcbdcfa2f0b480abae7e2526ddad75cf20f9becf1b
iteration 0: [8] -> [4]:
mt.root = commits[0] =  b'&\xfa\xe6q\x1f\x08h"%\xd2\x19\x9f\x82\x01\x83\x91]:\xfa\xa2<\x82\xa8ao\x8e\x16\xbeu\xa9\x9b\xd9'
update h_state <- H( h_state|| xi ): b'\xf8\xff?\x90]\xfe\x94?jM\n\x0cH\xba\x04nU\xd0\xed\xe5De3r<-n\x9fL\xd2R\xf6'
update i: 1
derive new challenge xi <- H(h_state||4||1) : 0xc2699f897fff266d98801881da3468244cf53838795df1ef
new polynomial length [4]
iteration 1: [4] -> [2]:
mt.root = commits[1] =  b'\xdb5\x96\xf6\x8e\x89\x16^\x8c\x05\x9d\xb6\xbe\xb9N\x06\x0c\xc3\x91M\x86\xcf\xd5\x961\xc2\x11\x99 =\xb3*'
update h_state <- H( h_state|| xi ): b'\x94:T\xbb\x9a\xf1W@\xb6\xee\xda\x17\x13\xa3\x7fW\xee\xde\x89\xb4\xd1m\x9bj)T\x1a\x0b\xa4\x02f\x9d'
update i: 2
derive new challenge xi <- H(h_state||5||1) : 0x60bf68e365f0747c6446e96830a1b8df0b15c2b29ab39427
new polynomial length [2]
update h_state <- H( xi || de

True